# MNIST & CNN - 컨볼루션 신경망을 이용해 MNIST 데이터셋 분류하기

## Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

## Device & reproducibility


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

## Parameters

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# Data & Data Loader


In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download= True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size= batch_size,
                                          shuffle= True,
                                          drop_last= True)

# Model

In [5]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size= 3, stride =1, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size= 3, stride= 1,padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.fc = nn.Linear(7*7*64, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out

In [6]:
model = CNN().to(device)

## Cost function & optimizer

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

## Training

In [8]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost / total_batch
  
  print('Epoch: {}  Cost= {}'.format(epoch+1, avg_cost))
print('Learning Finished')

Epoch: 1  Cost= 0.22563402354717255
Epoch: 2  Cost= 0.06299302726984024
Epoch: 3  Cost= 0.046312857419252396
Epoch: 4  Cost= 0.03758454695343971
Epoch: 5  Cost= 0.0313216969370842
Epoch: 6  Cost= 0.026079528033733368
Epoch: 7  Cost= 0.02169409580528736
Epoch: 8  Cost= 0.018283745273947716
Epoch: 9  Cost= 0.01641400158405304
Epoch: 10  Cost= 0.013174688443541527
Epoch: 11  Cost= 0.009898602031171322
Epoch: 12  Cost= 0.00999749731272459
Epoch: 13  Cost= 0.008744429796934128
Epoch: 14  Cost= 0.006396235898137093
Epoch: 15  Cost= 0.006209356710314751
Learning Finished


## Testing

In [9]:
with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)  
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9856999516487122
